# Batch variant scoring

In this notebook, we demonstrate how to score many variants using AlphaGenome.

To prepare numerous variants for scoring, provide the following information as
columns in a tab-separated Variant Call Format (VCF) file: - `variant_id`: A
unique identifier for each variant. - `CHROM`: Chromosome, specified as a string
beginning with 'chr' (e.g., 'chr1'). - `POS`: Integer representing the base pair
position (1-based; build hg38 (human) or mm10 (mouse) - see
[FAQ](https://www.alphagenomedocs.com/faqs.html#how-do-i-define-a-variant)). -
`REF`: The reference nucleotide sequence at the specified position. - `ALT`: The
alternate nucleotide sequence at the specified position.

``` {tip}
Open this tutorial in Google Colab for interactive viewing.
```

In [1]:
# @title Install AlphaGenome

# @markdown Run this cell to install AlphaGenome.
from IPython.display import clear_output
! pip install alphagenome
clear_output()

In [4]:
# @title Setup and imports.

from io import StringIO
from alphagenome import colab_utils
from alphagenome.data import genome
from alphagenome.models import dna_client, variant_scorers
from google.colab import data_table, files
import pandas as pd
from tqdm import tqdm

data_table.enable_dataframe_formatter()

# Load the model.
dna_model = dna_client.create(colab_utils.get_api_key())

In [8]:
vcf_all = pd.read_csv('sort.overall_chip_variants.tsv', sep='\t')
# vcf_all.head(20)

vcf_all[vcf_all['CHROM']=="chr22"].head(20)


,variant_id,CHROM,POS,REF,ALT
2743,chr22_28365160_C_T,chr22,28365160,C,T
2744,chr22_28695868_AG_A,chr22,28695868,AG,A
2745,chr22_28707610_T_C,chr22,28707610,T,C
2746,chr22_29020414_C_T,chr22,29020414,C,T
2747,chr22_29054205_G_A,chr22,29054205,G,A
2748,chr22_50532618_T_C,chr22,50532618,T,C


In [10]:
# @title Score batch of variants.

# Load VCF file containing variants.
# vcf_file = "'sort.overall_chip_variants.tsv'" # @param {"type":"string","placeholder":"sort.overall_chip_variants.tsv"}

# We provide an example list of variants to illustrate.
# vcf_file = """variant_id\tCHROM\tPOS\tREF\tALT
# chr3_58394738_A_T_b38\tchr3\t58394738\tA\tT
# chr8_28520_G_C_b38\tchr8\t28520\tG\tC
# chr16_636337_G_A_b38\tchr16\t636337\tG\tA
# chr16_1135446_G_T_b38\tchr16\t1135446\tG\tT
# """

## vcf = pd.read_csv(StringIO(vcf_file), sep='\t')
# vcf_file = 'sort.overall_chip_variants.tsv'
# vcf = pd.read_csv(vcf_file, sep='\t')

for chrom, vcf in vcf_all.groupby('CHROM'):

  print(f"Processing {chrom}")

  required_columns = ['variant_id', 'CHROM', 'POS', 'REF', 'ALT']
  for column in required_columns:
    if column not in vcf.columns:
      raise ValueError(f'VCF file is missing required column: {column}.')

  organism = 'human'  # @param ["human", "mouse"] {type:"string"}

# @markdown Specify length of sequence around variants to predict:
  sequence_length = '1MB'  # @param ["16KB", "100KB", "500KB", "1MB"] { type:"string" }
  sequence_length = dna_client.SUPPORTED_SEQUENCE_LENGTHS[
      f'SEQUENCE_LENGTH_{sequence_length}'
  ]

# @markdown Specify which scorers to use to score your variants:
  score_rna_seq = True  # @param { type: "boolean"}
  score_cage = True  # @param { type: "boolean" }
  score_procap = True  # @param { type: "boolean" }
  score_atac = True  # @param { type: "boolean" }
  score_dnase = True  # @param { type: "boolean" }
  score_chip_histone = True  # @param { type: "boolean" }
  score_chip_tf = True  # @param { type: "boolean" }
  score_polyadenylation = True  # @param { type: "boolean" }
  score_splice_sites = True  # @param { type: "boolean" }
  score_splice_site_usage = True  # @param { type: "boolean" }
  score_splice_junctions = True  # @param { type: "boolean" }

# @markdown Other settings:
  download_predictions = True  # @param { type: "boolean" }

# Parse organism specification.
  organism_map = {
      'human': dna_client.Organism.HOMO_SAPIENS,
      'mouse': dna_client.Organism.MUS_MUSCULUS,
  }
  organism = organism_map[organism]

# Parse scorer specification.
  scorer_selections = {
      'rna_seq': score_rna_seq,
      'cage': score_cage,
      'procap': score_procap,
      'atac': score_atac,
      'dnase': score_dnase,
      'chip_histone': score_chip_histone,
      'chip_tf': score_chip_tf,
      'polyadenylation': score_polyadenylation,
      'splice_sites': score_splice_sites,
      'splice_site_usage': score_splice_site_usage,
      'splice_junctions': score_splice_junctions,
  }

  all_scorers = variant_scorers.RECOMMENDED_VARIANT_SCORERS
  selected_scorers = [
      all_scorers[key]
      for key in all_scorers
      if scorer_selections.get(key.lower(), False)
  ]

# Remove any scorers or output types that are not supported for the chosen organism.
  unsupported_scorers = [
      scorer
      for scorer in selected_scorers
      if (
          organism.value
          not in variant_scorers.SUPPORTED_ORGANISMS[scorer.base_variant_scorer]
      )
      | (
          (scorer.requested_output == dna_client.OutputType.PROCAP)
          & (organism == dna_client.Organism.MUS_MUSCULUS)
      )
  ]
  if len(unsupported_scorers) > 0:
    print(
        f'Excluding {unsupported_scorers} scorers as they are not supported for'
        f' {organism}.'
    )
    for unsupported_scorer in unsupported_scorers:
      selected_scorers.remove(unsupported_scorer)


# Score variants in the VCF file.
  results = []

  for i, vcf_row in tqdm(vcf.iterrows(), total=len(vcf)):
    variant = genome.Variant(
        chromosome=str(vcf_row.CHROM),
        position=int(vcf_row.POS),
        reference_bases=vcf_row.REF,
        alternate_bases=vcf_row.ALT,
        name=vcf_row.variant_id,
    )
    interval = variant.reference_interval.resize(sequence_length)

    variant_scores = dna_model.score_variant(
        interval=interval,
        variant=variant,
        variant_scorers=selected_scorers,
        organism=organism,
    )
    results.append(variant_scores)

  df_scores = variant_scorers.tidy_scores(results)

  if download_predictions:
    output_file = f"variant_scores_{chrom}.csv.gz"
    df_scores.to_csv(output_file, index=False, compression='gzip')
    files.download(output_file)

#df_scores.head()


Processing chr1


100%|██████████| 65/65 [01:20<00:00,  1.23s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processing chr10


...

<IPython.core.display.Javascript object>

Processing chr9


100%|██████████| 1/1 [00:01<00:00,  1.09s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Note that the resulting output dataframe could be quite large, especially if you
use many scorers for scoring. Very large dataframes can't be filtered
interactively, but you can interact with them using the standard `pandas`
commands:

In [11]:
# Examine just the effects of the variants on T-cells.
columns = [c for c in df_scores.columns if c != 'ontology_curie']
df_scores[(df_scores['ontology_curie'] == 'CL:0000084')][columns]

,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,...,biosample_type,biosample_life_stage,data_source,endedness,genetically_modified,transcription_factor,histone_mark,gtex_tissue,raw_score,quantile_score
0,chr9:111887662:CAT>C,chr9:111363375-112411951:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,0.001990,0.011535
168,chr9:111887662:CAT>C,chr9:111363375-112411951:.,None,None,None,None,None,None,DNASE,"CenterMaskScorer(requested_output=DNASE, width...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,-0.003013,-0.034594
2109,chr9:111887662:CAT>C,chr9:111363375-112411951:.,None,None,None,None,None,None,CHIP_HISTONE,CenterMaskScorer(requested_output=CHIP_HISTONE...,...,primary_cell,adult,encode,single,False,NaN,H3K27ac,NaN,0.001052,0.023068
2110,chr9:111887662:CAT>C,chr9:111363375-112411951:.,None,None,None,None,None,None,CHIP_HISTONE,CenterMaskScorer(requested_output=CHIP_HISTONE...,...,primary_cell,adult,encode,single,False,NaN,H3K27me3,NaN,-0.007988,-0.584379
2111,chr9:111887662:CAT>C,chr9:111363375-112411951:.,None,None,None,None,None,None,CHIP_HISTONE,CenterMaskScorer(requested_output=CHIP_HISTONE...,...,primary_cell,adult,encode,single,False,NaN,H3K36me3,NaN,0.002559,0.248480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,chr9:111887662:CAT>C,chr9:111363375-112411951:.,ENSG00000230081,HSPE1P28,processed_pseudogene,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,primary_cell,adult,encode,single,False,NaN,NaN,,-0.006210,-0.985088
7609,chr9:111887662:CAT>C,chr9:111363375-112411951:.,ENSG00000242256,RN7SL57P,misc_RNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,primary_cell,adult,encode,single,False,NaN,NaN,,-0.086349,-0.999882
7610,chr9:111887662:CAT>C,chr9:111363375-112411951:.,ENSG00000243911,RN7SL430P,misc_RNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,primary_cell,adult,encode,single,False,NaN,NaN,,-0.037908,-0.999608
7611,chr9:111887662:CAT>C,chr9:111363375-112411951:.,ENSG00000259953,LINC02977,lncRNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,primary_cell,adult,encode,single,False,NaN,NaN,,0.067837,0.999840
